# TAXI TRIP DURATION 

## IMPORT LIBRARIES

In [2]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline

## SET UP MLFLOW

In [20]:
import mlflow 
import os 

os.environ["AWS_PROFILE"] = "MLOps-dev"

TRACKING_URI = "ec2-54-219-113-126.us-west-1.compute.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_URI}:5000")
mlflow.set_experiment("taxi_trip_prediction-experiment")

<Experiment: artifact_location='s3://mlflow-artifacts-prem/1', experiment_id='1', lifecycle_stage='active', name='taxi_trip_prediction-experiment', tags={}>

## DATA PREPARATION

In [21]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df['duration'].dt.total_seconds() / 60
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df
    
def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [22]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

## MODEL TRAINING

In [23]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


/Users/premsurawut/anaconda3/envs/course-mlops-zoomcamp/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## EXPORT MODEL

In [24]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'ec2-54-219-113-126.us-west-1.compute.amazonaws.com'
RUN_ID = '3fc5b94495d54d978b8dcb5094cccdcf'

client = MlflowClient(tracking_uri=f'http://{MLFLOW_TRACKING_URI}:5000')

In [25]:
path = client.download_artifacts(run_id=RUN_ID, path='model/model.pkl')

In [26]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [27]:
dv

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, min_samples_leaf=10,
                                       n_jobs=-1, random_state=0))])